In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.3.13
# LRU缓存策略：lintcode 134

In [1]:
class LinkNode:
    def __init__(self, key = None, value = None, prev = None, next = None):
        self.value = value
        self.key = key
        self.prev = prev
        self.next = next
        
## 哈希表+链表的实现
## get和set需要O(1)的时间复杂度
class LRUCache:
    """
    @param: capacity: An integer
    """
    ## 存储数据使用哈希表，这样可以快速找到某个值
    def __init__(self, capacity):
        # do intialization if necessary
        self.capacity = capacity
        ## 链表操作中，要设置以哨兵结点dummy，这结点永远指向头结点
        self.dummy = LinkNode()  ## 头指针
        self.tail = self.dummy  ## 尾指针
        ## 九章算法中存储key存储的是当前结点的前一个结点，原因是结点本身没有设置前驱结点，只有后继结点
        ## 由于后续操作需要使用结点的前驱结点，所以就把结点的前驱结点存放到哈希表中
        self.key_to_node = {}

    """
    @param: key: An integer
    @return: An integer
    """
    def get(self, key):
        # write your code here
        ## key不存在
        if key not in self.key_to_node:
            return -1
        
        ## 如果存在，把访问的结点挪动到尾部
        self.kick(self.key_to_node[key])
        return self.tail.value  ## 尾指针的value就是key对应的结点的value

    ## 尾部插入
    def push_back(self, node):
        ## self.tail指向的当前的尾部结点，但其本身不是链表中的结点
        node.prev = self.tail
        self.tail.next = node
        self.tail = node
        node.next = None
        
        ## node有可能在哈希表中，也有可能不在
        if node.key not in self.key_to_node:
            self.key_to_node[node.key] = node
    
    ## 头部删除
    def pop_first(self):
        ## dummy哨兵结点的next就是链表中的头结点
        first = self.dummy.next
        first.next.prev = self.dummy
        self.dummy.next = first.next
        del self.key_to_node[first.key]
    
    ## 将结点移动到尾部
    def kick(self, node):
        if node == self.tail:  ## 本身就是尾结点
            return
        
        ## 链表的基本操作
        prev = node.prev
        next = node.next
        ## 结点的前驱结点的后继结点要先指向node的后继结点
        ## 双向指针，前驱结点和后继结点都要修改
        prev.next = next
        next.prev = prev
        ## node插入到尾部
        self.push_back(node)
        
    """
    @param: key: An integer
    @param: value: An integer
    @return: nothing
    """
    ## LRU存储的数据需要有一定的顺序，因为容量超出时，就需要删除掉最近没有访问的元素
    ## 最近访问的元素应该在尾部，这样便于插入，每次插入的元素如果之前不存在，就直接插入
    ## 到尾部，如果之前已存在，则移动到尾部，这样尾部的元素就一直是最近访问的
    def set(self, key, value):
        # write your code here
        ## 如果key已存在
        if key in self.key_to_node:
            self.key_to_node[key].value = value  ## 修改值
            ## 移动到尾部
            self.kick(self.key_to_node[key])
            return

        ## 如果key不存在
        self.push_back(LinkNode(key, value))
        ## 如果容量超出限制，就删除头结点
        if len(self.key_to_node) > self.capacity:
            self.pop_first()
    
